# HW03
Deep Learning, GIST RT5101-01, 2024, Spring, (Tue/Thurs 2:30~3:45)
***

### Problem1. Build cat and dog detection dataset code
(Download URL: https://www.kaggle.com/datasets/andrewmvd/dog-and-cat-detection)
- Build detection dataset code
- Check your dataset code
----

### Problem2. Train cat and dog detection data using FasterRCNN
- Training detection dataset
- Check the result
- Analysis your result

***
### You can add additional code for checking your image and model.
### You must summit ``.ipynb`` file. Do not summit ``.py`` file.
---

### How to submit your homework
Submit your jupyter notebook file with the filename of  *HW03_studentnumber.ipynb*  on GIST LMS

Ex) HW03_20222015.ipynb  

### Submission deadline
2024.06.16, Sunday 23:59 (PM)

### Plagiarism
We encourage you to discuss this homework with your friends or TA, but you should write your own code.




***
***

In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Problem 1. (total 5 pt.)
**Build cat and dog detection dataset code**

- **1**. Build dataset code
- **2**. Split into train dataset and test dataset.
- **3**. Check your dataset code

### Problem 1-1. (2 pt.)
Build dataset code

In [ ]:
import os
from bs4 import BeautifulSoup

# This is a reference code for reading an XML file.
# Utilize this code to read an annotation file to extract bbox information.

def read_ann(directory):
    annotations = os.listdir(directory)

    records = []
    for annot_file in annotations:
        with open(os.path.join(directory,annot_file), 'r') as f:
            file = f.read()

        data = BeautifulSoup(file, "xml")

        objects = data.find_all('object')
        length = len(objects)
        filename = data.find("filename").text
        # img_width = int(data.find('size').width.text)
        # img_height = int(data.find('size').height.text)

        for i in range(length):
            class_name = data.find_all('object')[i].find('name').text
            xmin = int(data.find_all('object')[i].find('bndbox').find('xmin').text)#/img_width
            ymin = int(data.find_all('object')[i].find('bndbox').find('ymin').text)#/img_height
            xmax = int(data.find_all('object')[i].find('bndbox').find('xmax').text)#/img_width
            ymax = int(data.find_all('object')[i].find('bndbox').find('ymax').text)#/img_height

            records.append([filename,class_name,xmin,ymin,xmax,ymax])
    return records

In [ ]:

import os
import numpy as np
import torch
from PIL import Image

class CatDogDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms

        self.imgs = # image file name list.

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

### Problem 1-2. (2 pt.)
Split into train dataset and test dataset.


In [ ]:
batch_size =

In [ ]:
import sys
### Change to your own path
sys.path.append('/content/drive/MyDrive/2024_deep_learning/coding_lecture/04.04.CodingLecture05/vis_utils')

import transforms as T

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))
# used for padding variable-length batches.
# "*" = splat operator : unpacking a list into arguments

In [ ]:
def get_transform():
  transforms = T.Compose([
      T.ToTensor(),
  ])

  return transforms

In [ ]:
data_loader =
data_loader_test =

In [ ]:
# Check dataset loader's return
for i, (img, target) in enumerate(data_loader):
  for j in range(batch_size):
    print('SIze of image : (C, H, W) ',img[j].shape)
    print('Number of the objects :', len(target[j]['boxes']))
  if i> 3:
    break

### Problem 1-3. (1 pt.)
Check your dataset code

In [ ]:
# visualize train image

In [ ]:
# visualize train image's bbox

---
---

## Problem 2. (Total 5 pt.)
- **Train Dogs and Cats data using FasterRCNN**


### Problem 2-1. (3 pt.)
- **Step 1**. Import package.
- **Step 2**. Build your FasterRCNN model.
- **Step 3**. Define hyper-parameter for training
- **Step 4**. Train the model and check the validation results  

In [ ]:
''' Step 1 '''
# Import package.

In [ ]:
import sys
### Change to your own path
sys.path.append('/content/drive/MyDrive/2024_deep_learning/coding_lecture/04.04.CodingLecture05/vis_utils')

import utils
from engine import *
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

In [ ]:
''' Step 2 '''
# Build your FasterRCNN model.

model =

# change classififer class size


In [ ]:
''' Step 3 '''
# Define hyper-parameter for training.

params =

optimizer =

lr_scheduler =

num_epochs =

In [ ]:
''' Step 4 '''
# Train the model and check the validation results

In [ ]:
# define model training function

In [ ]:
# model training
for epoch in range(num_epochs):

In [ ]:
# define model evaluate function

In [ ]:
# run evaluate code

### Problem 2-2. (2 pt.)
Analysis your result

In [ ]:
import cv2
def overlay_instances(img, prediction, threshold=0.8):
    label_dict = {0: 'cat', 1: 'dog'}
    ori_img = img.mul(255).permute(1,2,0).byte().numpy() # Tensor * 255, Convert RGB -> BGR, Tensor to numpy
    for idx in range(len(prediction[0]['boxes'])):
        score = prediction[0]['scores'][idx].cpu().detach().numpy().item() # detach() : Generation of tensors that do not propagate gradients from existing sensors
        if score < threshold:
            continue
        box = x1,y1,x2,y2 = prediction[0]['boxes'][idx].cpu().detach().numpy()
        label = prediction[0]['labels'][idx].cpu().detach().numpy().item()
        print(x1,y1, x2, y2)
        cv2.rectangle(ori_img, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
        cv2.rectangle(ori_img, (int(x1), int(y1)), (int(x1+60), int(y1+20)), (255,0,0),-1)
        cv2.putText(ori_img, label_dict[label], (int(x1), int(y1+15)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), thickness=1)
    return Image.fromarray(ori_img)

In [ ]:
# Inference on test images

#### Your Anwser :